<a href="https://colab.research.google.com/github/arpadikuma/Colab_Projects/blob/main/ZTM_TF_DEV_2023_exercises_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXERCISES for ZTM TensorFlow Developer Certificate

## 08. Introduction to NLP (Natural Language Processing)


### Tasks

1. Rebuild, compile and train model_1, model_2 and model_5 using the Keras Sequential API instead of the Functional API.
2. Retrain the baseline model with 10% of the training data. How does perform compared to the Universal Sentence Encoder model with 10% of the training data?
3. Try fine-tuning the TF Hub Universal Sentence Encoder model by setting training=True when instantiating it as a Keras layer.
 >We can use this encoding layer in place of our text_vectorizer and embedding layer<br>
 `sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=True)`
4. Retrain the best model you've got so far on the whole training set (no validation split). Then use this trained model to make predictions on the test dataset and format the predictions into the same format as the sample_submission.csv file from Kaggle (see the Files tab in Colab for what the sample_submission.csv file looks like). Once you've done this, make a submission to the Kaggle competition, how did your model perform?
5. Combine the ensemble predictions using the majority vote (mode), how does this perform compare to averaging the prediction probabilities of each model?
6. Make a confusion matrix with the best performing model's predictions on the validation set and the validation ground truth labels.

##### Necessary modules and libraries

In [ ]:
import os
import pathlib
import random
import datetime
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import matplotlib.pyplot as plt

print(tf.__version__)

2023-07-12 02:56:53.256614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0


In [ ]:
import sys

print("This notebook is connected to Google Colab" if "google.colab" in sys.modules else "This Notebook is connected to a local runtime")

This Notebook is connected to a local runtime


In [ ]:
if 'google.colab' in sys.modules:
    print("This Notebook is running in Google Colab\n")
    local = False
    if os.path.isdir("/content/drive/MyDrive/TF_sources"):
        print("Connected to Google Drive through Google Colab\n")
        runtime = 3
        %cd /content/drive/MyDrive/TF_sources
        %ls
else:
    print("This Notebook is connected to a local runtime\n")
    local = True
    if os.path.isdir("C:\\Users\\ikuma\\OneDrive\\python\\datascience\\ZTM_Tensorflow\\NLP_TF"):
        print("Connected to native Windows Jupyter runtime\n")
        runtime = 1
        %cd C:\Users\ikuma\OneDrive\python\datascience\ZTM_Tensorflow\NLP_TF
        %ls
    elif os.path.isdir("/tf_gpu/ZTM_Tensorflow/"):
        print("Connected to WSL2 Docker Jupyter runtime\n")
        runtime = 2
        %cd /tf_gpu/ZTM_Tensorflow/NLP_TF/
        %ls
    else:
        print(".")

This Notebook is connected to a local runtime

Connected to WSL2 Docker Jupyter runtime

/tf_gpu/ZTM_Tensorflow/NLP_TF
docker_test/             nlp_getting_started.zip*  test.csv*
embedding_metadata.tsv*  sample_submission.csv*    train.csv*
embedding_vectors.tsv*   tensorboard_data/


In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Dataset

In [ ]:
# import zipfile

# zip_ref = zipfile.ZipFile("nlp_getting_started.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train_df["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [ ]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [ ]:
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print(f"---\n")

Target: 1 (real disaster)
Text:
nbanews Soudelor Typhoon Soudelor is taking dead aim at Taiwan and is expected to make landfall Friday according to the Joint TyphoonÛ_

---

Target: 1 (real disaster)
Text:
On 1st August #Turkish jets bombed the village Zergele in Qendil and killed 8 civilians and 15 others wounded

---

Target: 1 (real disaster)
Text:
Firefighters from Connecticut are headed to California to fight wild fires http://t.co/QWpUxPyWbF http://t.co/8jlXZ6fkxy

---

Target: 1 (real disaster)
Text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/pTeVY815mt

---

Target: 0 (not real disaster)
Text:
Kevin Tan says hail to the chefs - Û_ and Green Pastures which features sustainable and organic cooking.... http://t.co/D9xVuvp9s6

---



#### Data Preparation

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

#### Vectorization

In [ ]:
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
RT: ZONEWolf123: I liked a YouTube video http://t.co/u66kYg11ZD Minecraft: NIGHT LUCKY BLOCK MOD (BOB APOCALYPSE WITHER 2.0 &amp; MORE!) MoÛ_


Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  96, 6474,    8,  317,    3,  114,   72,    1, 1811,  285, 1276,
        1549, 1120, 2538,  409]])>

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#### Embedding

In [ ]:
tf.keras.utils.set_random_seed(42)

from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length,
                             name="embedding_1")

embedding

In [ ]:
random_sentence = random.choice(train_sentences)
print(f"Original sentence:\n{random_sentence}\n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentence:
You are listening to LLEGASTE TU - TWISTER EL REY

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01463007, -0.02484524, -0.04815755, ...,  0.00668095,
         -0.04589608,  0.04178235],
        [-0.04285816,  0.02968811, -0.02570785, ..., -0.01790828,
          0.00757997,  0.00572492],
        [-0.03834195, -0.04880849,  0.04038361, ..., -0.00352918,
         -0.0356357 ,  0.00734928],
        ...,
        [ 0.03581891,  0.01626286, -0.01412892, ..., -0.02094598,
          0.03622821, -0.0372517 ],
        [ 0.03581891,  0.01626286, -0.01412892, ..., -0.02094598,
          0.03622821, -0.0372517 ],
        [ 0.03581891,  0.01626286, -0.01412892, ..., -0.02094598,
          0.03622821, -0.0372517 ]]], dtype=float32)>

In [ ]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 1.4630068e-02, -2.4845243e-02, -4.8157550e-02,  1.3964776e-02,
       -1.4338959e-02,  1.2196850e-02,  3.7065659e-02, -6.9518089e-03,
       -2.9642273e-02,  3.1561900e-02,  1.9796267e-03,  1.5800212e-02,
       -4.7469068e-02, -1.9296432e-02, -3.5913657e-02, -8.1322789e-03,
       -3.1712342e-02,  3.6084507e-02,  2.0850111e-02, -4.9872398e-02,
       -1.6938247e-02, -1.1862397e-02,  3.3234764e-02,  2.7567256e-02,
       -1.5823983e-02,  3.2850232e-02, -4.1784644e-03,  2.6826207e-02,
        2.5103662e-02, -1.1744369e-02,  3.4217704e-02, -4.0949695e-03,
        8.8067278e-03, -5.5586100e-03, -3.8908076e-02, -4.9307622e-02,
        2.2960160e-02, -5.6890398e-04,  1.8283043e-02, -1.6368411e-02,
       -4.1949928e-02, -3.0839253e-02, -4.3374695e-02,  2.6200403e-02,
        4.9974550e-02, -4.4647910e-02,  1.8289413e-02,  3.4918394e-02,
       -2.8094877e-02,  2.1710310e-02,  1.1039685e-02, -1.9944334e-02,
        2.3281697e-02, -1.036

#### Rebuild, compile and train model_1, model_2 and model_5 using the Keras Sequential API instead of the Functional API.

##### Model 1: feed-forward neural network (dense model)

In [ ]:
def create_tensorboard_callback(experiment_name, dir_name="tensorboard_data"):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

In [ ]:
tf.keras.utils.set_random_seed(42)
model_1 = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype="string"),
    text_vectorizer,
    embedding,
    layers.GlobalAveragePooling1D(),
    layers.Dense(1, activation="sigmoid")
], name="nlp_dense")

In [ ]:
model_1.compile(optimizer=tf.keras.optimizers.Adam(),
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [ ]:
model_1.summary()

Model: "nlp_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_5   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(model_1.name)])

Saving TensorBoard log files to: tensorboard_data/nlp_dense/20230627-023333
Epoch 1/5
215/215 [==============================] - 2s 6ms/step - loss: 0.4069 - accuracy: 0.8860 - val_loss: 0.4656 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2869 - accuracy: 0.9012 - val_loss: 0.4591 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2353 - accuracy: 0.9155 - val_loss: 0.4755 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2016 - accuracy: 0.9296 - val_loss: 0.4961 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1755 - accuracy: 0.9396 - val_loss: 0.5186 - val_accuracy: 0.7874


In [ ]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.5276 - accuracy: 0.7743


[0.5275543928146362, 0.7742782235145569]

In [ ]:
results_1 = model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4773 - accuracy: 0.7874


In [ ]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 1.8717017e-02,  3.3825781e-02, -3.1222217e-02, ...,
         -3.4147352e-03,  1.9617090e-02, -5.3262234e-02],
        [ 7.9920636e-03, -3.0470278e-02,  2.3969425e-02, ...,
         -3.8290016e-02,  3.6291275e-02,  4.5402542e-02],
        [-1.0398817e-02, -1.1811879e-02,  5.8957929e-05, ...,
          4.6165057e-02, -4.9918327e-02, -2.5232826e-04],
        ...,
        [ 6.3637607e-03, -1.3468124e-02,  4.7988441e-02, ...,
         -1.4667142e-02,  1.8839668e-02, -9.0965740e-03],
        [-5.0784964e-02,  2.6956283e-02, -1.0618210e-02, ...,
          4.4774819e-02, -9.3725011e-02, -8.6453453e-02],
        [-9.7745143e-02,  5.0630890e-02, -5.9932407e-02, ...,
          1.9838352e-02, -4.7273044e-02, -5.3474914e-02]], dtype=float32)>]

In [ ]:
embed_weights = model_1.get_layer('embedding_1').get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [ ]:
# !tensorboard dev upload --logdir ./tensorboard_data \
#     --name "NLP Dense model 1" \
#     --description "NLP exercise deep learning model with embedding layer" \
#     --one_shot

In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.4147926 ],
       [0.7440461 ],
       [0.9978345 ],
       [0.10640857],
       [0.11603917],
       [0.9324824 ],
       [0.91342604],
       [0.9928422 ],
       [0.96947104],
       [0.26506644]], dtype=float32)

In [ ]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred) * 100

    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}

    return model_results

In [ ]:
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [ ]:
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
model_1.summary()

Model: "nlp_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(embed_weights.shape)

(10000, 128)


In [ ]:
print(model_1.get_layer("embedding_1").get_weights()[0].shape)

(10000, 128)


In [ ]:
model_1.layers[1].weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 1.8717017e-02,  3.3825781e-02, -3.1222217e-02, ...,
         -3.4147352e-03,  1.9617090e-02, -5.3262234e-02],
        [ 7.9920636e-03, -3.0470278e-02,  2.3969425e-02, ...,
         -3.8290016e-02,  3.6291275e-02,  4.5402542e-02],
        [-1.0398817e-02, -1.1811879e-02,  5.8957929e-05, ...,
          4.6165057e-02, -4.9918327e-02, -2.5232826e-04],
        ...,
        [ 6.3637607e-03, -1.3468124e-02,  4.7988441e-02, ...,
         -1.4667142e-02,  1.8839668e-02, -9.0965740e-03],
        [-5.0784964e-02,  2.6956283e-02, -1.0618210e-02, ...,
          4.4774819e-02, -9.3725011e-02, -8.6453453e-02],
        [-9.7745143e-02,  5.0630890e-02, -5.9932407e-02, ...,
          1.9838352e-02, -4.7273044e-02, -5.3474914e-02]], dtype=float32)>]

In [ ]:
model_1.layers[1].get_weights()

[array([[ 1.8717017e-02,  3.3825781e-02, -3.1222217e-02, ...,
         -3.4147352e-03,  1.9617090e-02, -5.3262234e-02],
        [ 7.9920636e-03, -3.0470278e-02,  2.3969425e-02, ...,
         -3.8290016e-02,  3.6291275e-02,  4.5402542e-02],
        [-1.0398817e-02, -1.1811879e-02,  5.8957929e-05, ...,
          4.6165057e-02, -4.9918327e-02, -2.5232826e-04],
        ...,
        [ 6.3637607e-03, -1.3468124e-02,  4.7988441e-02, ...,
         -1.4667142e-02,  1.8839668e-02, -9.0965740e-03],
        [-5.0784964e-02,  2.6956283e-02, -1.0618210e-02, ...,
          4.4774819e-02, -9.3725011e-02, -8.6453453e-02],
        [-9.7745143e-02,  5.0630890e-02, -5.9932407e-02, ...,
          1.9838352e-02, -4.7273044e-02, -5.3474914e-02]], dtype=float32)]

In [ ]:
model_1.layers[1].weights[0].shape

TensorShape([10000, 128])

In [ ]:
import io

out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

for num, word in enumerate(words_in_vocab):
    if num == 0:
        continue
    vec = embed_weights[num]
    out_m.write(word+"\n")
    out_v.write("\t".join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download("embedding_vectors.tsv")
    files.download("embedding_metadata.tsv")

##### Model 2: LSTM

In [ ]:
tf.keras.utils.set_random_seed(42)

model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")

model_2 = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype="string"),
    text_vectorizer,
    model_2_embedding,
    layers.LSTM(64),
    layers.Dense(1, activation="sigmoid")
], name="nlp_LSTM")

In [ ]:
model_2.compile(optimizer=tf.keras.optimizers.Adam(),
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [ ]:
model_2.summary()

Model: "nlp_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(model_2.name)])

Saving TensorBoard log files to: tensorboard_data/nlp_LSTM/20230626-041256
Epoch 1/5
215/215 [==============================] - 7s 12ms/step - loss: 0.5126 - accuracy: 0.7414 - val_loss: 0.4579 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3175 - accuracy: 0.8705 - val_loss: 0.5095 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2193 - accuracy: 0.9169 - val_loss: 0.5887 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1556 - accuracy: 0.9451 - val_loss: 0.6210 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1107 - accuracy: 0.9600 - val_loss: 0.8193 - val_accuracy: 0.7625


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


((762, 1),
 array([[0.02031598],
        [0.7857919 ],
        [0.99923456],
        [0.08825447],
        [0.00357198],
        [0.9990928 ],
        [0.95157886],
        [0.99965847],
        [0.99944514],
        [0.24452403]], dtype=float32))

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 76.24671916010499,
 'precision': 0.7637525216162191,
 'recall': 0.7624671916010499,
 'f1': 0.7603795119224996}

##### Model 3: GRU (gated recurrent unit)

In [ ]:
tf.keras.utils.set_random_seed(42)

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

model_3 = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype="string"),
    text_vectorizer,
    model_3_embedding,
    layers.GRU(64),
    layers.Dense(1, activation="sigmoid")
], name="nlp_GRU")

In [ ]:
model_3.compile(optimizer=tf.keras.optimizers.Adam(),
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [ ]:
model_3.summary()

Model: "nlp_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_3_history = model_3.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(model_3.name)])

Saving TensorBoard log files to: tensorboard_data/nlp_GRU/20230626-041311
Epoch 1/5
215/215 [==============================] - 4s 11ms/step - loss: 0.5276 - accuracy: 0.7243 - val_loss: 0.4575 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3192 - accuracy: 0.8692 - val_loss: 0.4924 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2165 - accuracy: 0.9191 - val_loss: 0.5643 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1544 - accuracy: 0.9466 - val_loss: 0.6257 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1173 - accuracy: 0.9597 - val_loss: 0.5996 - val_accuracy: 0.7703


In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[0.37191686],
        [0.9182355 ],
        [0.99688184],
        [0.16246006],
        [0.01183802],
        [0.98987764],
        [0.6975621 ],
        [0.9966595 ],
        [0.99608946],
        [0.3310669 ]], dtype=float32))

In [ ]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_3_results = calculate_results(val_labels, model_3_preds)
model_3_results

{'accuracy': 77.03412073490814,
 'precision': 0.7708102089179978,
 'recall': 0.7703412073490814,
 'f1': 0.7689180260110319}

In [ ]:
model_2_results

{'accuracy': 76.24671916010499,
 'precision': 0.7637525216162191,
 'recall': 0.7624671916010499,
 'f1': 0.7603795119224996}

In [ ]:
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

##### Model 5: Conv1D

In [ ]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation="relu")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [ ]:
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.00655079, -0.00130182, -0.03378217, ...,  0.03269541,
           0.0326734 , -0.02459557],
         [-0.01079181,  0.03284334, -0.01144771, ...,  0.06684816,
          -0.00707625, -0.05762339],
         [ 0.0022808 ,  0.01453666, -0.02588522, ...,  0.05281382,
          -0.03097097,  0.01772211],
         ...,
         [ 0.01871702,  0.03382578, -0.03122222, ..., -0.00341474,
           0.01961709, -0.05326223],
         [ 0.01871702,  0.03382578, -0.03122222, ..., -0.00341474,
           0.01961709, -0.05326223],
         [ 0.01871702,  0.03382578, -0.03122222, ..., -0.00341474,
           0.01961709, -0.05326223]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[0.        , 0.        , 0.09083761, 0.        , 0.09395183,
          0.02685726, 0.0372438 , 0.        , 0.        , 0.        ,
          0.06272615, 0.        , 0.02816107, 0.        , 0.06451887,
          0.        , 0.

In [ ]:
tf.keras.utils.set_random_seed(42)

model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")

model_5 = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype="string"),
    text_vectorizer,
    model_5_embedding,
    layers.Conv1D(filters=32, kernel_size=5, activation="relu"),
    layers.GlobalMaxPool1D(),
    layers.Dense(1, activation="sigmoid")
], name="nlp_conv1d")

In [ ]:
model_5.compile(optimizer=tf.keras.optimizers.Adam(),
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [ ]:
model_5.summary()

Model: "nlp_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,300,545
Trainable params: 1,300,545
Non-

In [ ]:
model_5_history = model_5.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(model_5.name)])

Saving TensorBoard log files to: tensorboard_data/nlp_conv1d/20230626-041325
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.5666 - accuracy: 0.7174 - val_loss: 0.4725 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3445 - accuracy: 0.8570 - val_loss: 0.4700 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2161 - accuracy: 0.9187 - val_loss: 0.5347 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1401 - accuracy: 0.9542 - val_loss: 0.6088 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0985 - accuracy: 0.9669 - val_loss: 0.6654 - val_accuracy: 0.7664


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.56074655],
       [0.67379695],
       [0.99976724],
       [0.06960978],
       [0.00621164],
       [0.9958936 ],
       [0.9753998 ],
       [0.99725014],
       [0.99883765],
       [0.5432859 ]], dtype=float32)

In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
model_5_results = calculate_results(val_labels, model_5_preds)
model_5_results

{'accuracy': 76.64041994750657,
 'precision': 0.7674517658976838,
 'recall': 0.7664041994750657,
 'f1': 0.7645419362124183}

#### 2. Baseline Model With 10% Training Data.

 How does it perform compared to the Universal Sentence Encoder model with 10% of the training data

In [ ]:
# create 10% subset of training data
train_10percent = train_df_shuffled[["text", "target"]].sample(frac=0.1)
train_sentences_10p = train_10percent["text"].to_list()
train_labels_10p = train_10percent["target"].to_list()

len(train_sentences_10p), len(train_labels_10p)

(761, 761)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(X=train_sentences_10p, y=train_labels_10p)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
baseline_score_10p = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score_10p*100:.2f}% on 10% of training data")

Our baseline model achieves an accuracy of: 93.96% on 10% of training data


In [ ]:
baseline_preds_10p = model_0.predict(val_sentences)
baseline_preds_10p[:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [ ]:
baseline_results_10p = calculate_results(val_labels, baseline_preds_10p)
baseline_results_10p

{'accuracy': 93.96325459317585,
 'precision': 0.9431798909751665,
 'recall': 0.9396325459317585,
 'f1': 0.9392438161138947}

results of 10% training data with Universal Sentence Encoder:

>'accuracy': 77.03412073490814,<br>
 'precision': 0.7760118694840564,<br>
 'recall': 0.7703412073490814,<br>
 'f1': 0.7665375100103654

#### Fine-Tuning TF Hub Universal Sentence Encoder Model

We can use this encoding layer in place of our text_vectorizer and embedding layer<br>
>sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
import tensorflow_hub as hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [ ]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="nlp_tfhub_1")

In [ ]:
model_6.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [ ]:
model_6.summary()

Model: "nlp_tfhub_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 256,830,721
Non-trainable params: 0
_________________________________________________________________


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2,
                                                 patience=2,
                                                 verbose=1,
                                                 min_lr=1e-7)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3)

In [ ]:
with tf.device("/CPU:0"):
    model_6_history = model_6.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(model_6.name)])

Saving TensorBoard log files to: tensorboard_data/nlp_tfhub_1/20230626-122820
Epoch 1/5
215/215 [==============================] - 27s 102ms/step - loss: 0.5859 - accuracy: 0.7405 - val_loss: 0.4957 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 22s 101ms/step - loss: 0.4332 - accuracy: 0.8159 - val_loss: 0.4364 - val_accuracy: 0.8176
Epoch 3/5
215/215 [==============================] - 21s 98ms/step - loss: 0.3582 - accuracy: 0.8516 - val_loss: 0.4200 - val_accuracy: 0.8228
Epoch 4/5
215/215 [==============================] - 21s 98ms/step - loss: 0.2882 - accuracy: 0.8847 - val_loss: 0.4238 - val_accuracy: 0.8281
Epoch 5/5
215/215 [==============================] - 21s 99ms/step - loss: 0.2126 - accuracy: 0.9203 - val_loss: 0.4396 - val_accuracy: 0.8255


In [ ]:
with tf.device("/CPU:0"):
    model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 12ms/step


array([[0.157929  ],
       [0.85108495],
       [0.9827121 ],
       [0.0809674 ],
       [0.9584237 ],
       [0.9636681 ],
       [0.98109484],
       [0.9840016 ],
       [0.9775354 ],
       [0.05330851]], dtype=float32)

In [ ]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 82.54593175853019,
 'precision': 0.8252919048410972,
 'recall': 0.8254593175853019,
 'f1': 0.82520208407303}

In [ ]:
import re
import os
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from transformers import AutoTokenizer

In [ ]:
bert_path = "https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
tokens = tokenizer.encode_plus("hello world", max_length=max_length,
                               truncation=True,
                               padding="max_length",
                               add_special_tokens=True,
                               return_token_type_ids=False,
                               return_attention_mask=True,
                               return_tensors="tf")

In [ ]:
tokens

{'input_ids': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[  101, 19082,  1362,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>}

In [ ]:
cv = 5
clf = LinearSVC()

model_x = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LinearSVC())
])

model_x.fit(X=train_sentences, y=train_labels)

# cross_val_score(clf, train_sentences, train_labels, scoring="accuracy", cv=cv)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [ ]:
model_x.score(val_sentences, val_labels)

0.7887139107611548

#### 4. Retrain The Best Model On The Whole Dataset (no validation split).

Use this trained model to make predictions on the test dataset and format the predictions into the same format as the sample_submission.csv file from Kaggle (see the Files tab in Colab for what the sample_submission.csv file looks like). Once you've done this, make a submission to the Kaggle competition, how did your model perform?

In [ ]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [ ]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [ ]:
%cd NLP_TF/

/tf_data/NLP_TF


In [ ]:
%ls -a

./  ../  .ipynb_checkpoints/  Untitled.ipynb


In [ ]:
!nvidia-smi

Thu Jun 29 03:31:56 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.50                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070 S...    On | 00000000:2D:00.0  On |                  N/A |
|  0%   52C    P8               23W / 235W|   1665MiB /  8192MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%ls -a

./
../
.empty/
.ipynb_checkpoints/
Amazon_Hackathon.ipynb*
P12-Churn-Modelling.xlsx*
P12-OfficeSupplies.csv*
ZTM_Business_Data_Analytics_Intelligence/
ZTM_TF_DEV_2023_exercises.ipynb*
ZTM_Tensorflow/
cnn/
credit_risk/
energy/
env/
env2/
environment.yml*
environment.yml.bak*
envtf/
kaggle/
model_checkpoints/
projects/
real_powercons_Germany_20190101_202301012359.xlsx*
saved_models/
tensorboard_data/


In [ ]:
%cd ..

/
